In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
import mediapipe

In [11]:
knuckle_model = tf.keras.models.load_model('../Training_Model1_23_6_10')
df = pd.read_csv('all_hands_with_landmark.csv')

In [12]:
df.head(4)

,Unnamed: 0.1,Unnamed: 0,path,label,landmarks
0,0,0,Hand_Images/1/63.jpg,1,"[[0.7918168306350708, 0.7815876007080078], [0...."
1,1,1,Hand_Images/1/189.jpg,1,"[[0.5782292485237122, 0.8176587224006653], [0...."
2,2,2,Hand_Images/1/77.jpg,1,"[[0.8111778497695923, 0.8097538948059082], [0...."
3,3,3,Hand_Images/1/162.jpg,1,"[[0.5877158045768738, 0.821674108505249], [0.3..."


In [13]:
import ast
data = df['landmarks'].apply(ast.literal_eval)

In [14]:
label = df['label']
label.head(4)

0    1
1    1
2    1
3    1
Name: label, dtype: int64

In [15]:
new_data = []
for i in range(len(data)):
    # print(type(data[i]))
    # print(data[i])
    temp = []
    for landmark in data[i]:
        for ele in landmark:
            temp.append(ele)
    new_data.append(temp)
data = new_data

In [19]:
result = []
for landmark in data:
    norm_landmarks = landmark
    norm_landmarks = np.array(norm_landmarks)
    landmark_tensor = tf.convert_to_tensor(norm_landmarks.reshape(1, 42), dtype=tf.float32)
    landmark_tensor = tf.expand_dims(landmark_tensor, axis=0)
    # predict
    prediction = knuckle_model.predict(landmark_tensor)
    result.append(np.argmax(prediction))

1/1 [==============================] - 0s 15ms/step


2023-06-10 19:59:19.467615: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 14ms/step


In [20]:
result[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [21]:
# combine result and label into df
result_df = pd.DataFrame(result, columns=['result'])
result_df['label'] = label
result_df

,result,label
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
1966,8,8
1967,8,8
1968,8,8
1969,8,8


In [22]:
# calculate accuracy
result_df['result'] = result_df['result'].astype('int')
result_df['label'] = result_df['label'].astype('int')
# compare result and label in one value
result_df['compare'] = result_df['result'] == result_df['label']
# calculate accuracy
accuracy = result_df['compare'].sum() / len(result_df)
accuracy


0.8472856418061897